In [10]:
# Install the Cassandra python driver
!pip install cassandra-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 64.8 MB/s eta 0:00:00


In [11]:
# Import the necessary libraries
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

In [12]:
# This secure connect bundle is autogenerated when you download your SCB,
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-my-work.zip'
}

# This token JSON file is autogenerated when you download your token,
# if yours is different update the file name below
with open("my_work-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

if session:
  print('Connected!')
else:
  print("An error occurred.")

Connected!


In [13]:
session.set_keyspace('example')  # Replace 'your_keyspace' with the correct name
print(f"Connected to keyspace: {session.keyspace}")


Connected to keyspace: example


In [14]:
rows = session.execute("SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'example'")
print("Existing tables:")
for row in rows:
    print(row.table_name)


Existing tables:


In [15]:
# Get a list of all tables in the keyspace
rows = session.execute("SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'example'")

# Drop each table
for row in rows:
    table_name = row.table_name
    try:
        session.execute(f"DROP TABLE IF EXISTS {table_name};")
        print(f"Table {table_name} dropped successfully.")
    except Exception as e:
        print(f"Error dropping table {table_name}: {e}")


In [16]:
session.execute("""
CREATE TABLE IF NOT EXISTS sales_bronze (
    region TEXT,
    country TEXT,
    item_type TEXT,
    sales_channel TEXT,
    order_priority TEXT,
    order_date TEXT,
    order_id UUID PRIMARY KEY,
    ship_date TEXT,
    units_sold INT,
    unit_price FLOAT,
    unit_cost FLOAT,
    total_revenue FLOAT,
    total_cost FLOAT,
    total_profit FLOAT
);
""")
print("Table sales_bronze created successfully.")


Table sales_bronze created successfully.


In [24]:
df = pd.read_csv('/content/sales_100 (1).csv')  # Replace with the actual file name after upload


In [25]:
import pandas as pd
import uuid

# Load data from CSV
df = pd.read_csv('/content/sales_100 (1).csv')

# Insert data into the Bronze table
for _, row in df.iterrows():
    session.execute("""
        INSERT INTO sales_bronze (region, country, item_type, sales_channel, order_priority, order_date, order_id, ship_date, units_sold, unit_price, unit_cost, total_revenue, total_cost, total_profit)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['Region'], row['Country'], row['Item Type'], row['Sales Channel'],
        row['Order Priority'], row['Order Date'], uuid.uuid4(),
        row['Ship Date'], row['UnitsSold'], row['UnitPrice'],
        row['UnitCost'], row['TotalRevenue'], row['TotalCost'], row['TotalProfit']
    ))
print("Data inserted into sales_bronze.")


Data inserted into sales_bronze.


In [26]:
rows = session.execute("SELECT * FROM sales_bronze LIMIT 10;")
for row in rows:
    print(row)


Row(order_id=UUID('d05750ff-3595-4aeb-a0c6-25bc2f0610d3'), country='Uganda', item_type='Cosmetics', order_date='2/28/2015', order_priority='M', region='Sub-Saharan Africa', sales_channel='Online', ship_date='3/15/2015', total_cost=1588143.25, total_profit=1048610.0, total_revenue=2636753.25, unit_cost=263.3299865722656, unit_price=437.20001220703125, units_sold=6031)
Row(order_id=UUID('e0620cbf-ae2c-4e26-adec-57b73ea99614'), country='Burundi', item_type='Beverages', order_date='3/9/2011', order_priority='H', region='Sub-Saharan Africa', sales_channel='Online', ship_date='4/9/2011', total_cost=112314.0703125, total_profit=55326.78125, total_revenue=167640.84375, unit_cost=31.790000915527344, unit_price=47.45000076293945, units_sold=3533)
Row(order_id=UUID('f98b5059-660a-4e1d-b38e-95549e6cfd1d'), country='Tanzania', item_type='Cosmetics', order_date='5/23/2016', order_priority='L', region='Sub-Saharan Africa', sales_channel='Offline', ship_date='5/24/2016', total_cost=2045547.5, total_pr

In [45]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/sales_100 (1).csv')  # Replace with the correct path to your file

# Check for missing (null) values
null_values = df.isnull().sum()

# Print the columns with missing values
print(null_values)


Region            0
Country           0
Item Type         0
Sales Channel     0
Order Priority    0
Order Date        0
Order ID          0
Ship Date         0
UnitsSold         0
UnitPrice         0
UnitCost          0
TotalRevenue      0
TotalCost         0
TotalProfit       0
dtype: int64


In [27]:
session.execute("""
CREATE TABLE IF NOT EXISTS sales_silver (
    order_id UUID PRIMARY KEY,
    region TEXT,
    country TEXT,
    item_type TEXT,
    total_profit FLOAT
);
""")
print("Table sales_silver created successfully.")


Table sales_silver created successfully.


In [ ]:
rows = session.execute("SELECT order_id, region, country, item_type, total_profit FROM sales_bronze;")
for row in rows:
    session.execute("""
        INSERT INTO sales_silver (order_id, region, country, item_type, total_profit)
        VALUES (%s, %s, %s, %s, %s)
    """, (row.order_id, row.region, row.country, row.item_type, row.total_profit))

print("Data successfully inserted into sales_silver.")


In [28]:
session.execute("""
CREATE TABLE IF NOT EXISTS sales_silver (
    order_id UUID PRIMARY KEY,
    region TEXT,
    country TEXT,
    item_type TEXT,
    total_profit FLOAT
);
""")
print("Table sales_silver created successfully.")


Table sales_silver created successfully.


In [29]:
rows = session.execute("SELECT order_id, region, country, item_type, total_profit FROM sales_bronze;")
for row in rows:
    session.execute("""
        INSERT INTO sales_silver (order_id, region, country, item_type, total_profit)
        VALUES (%s, %s, %s, %s, %s)
    """, (row.order_id, row.region, row.country, row.item_type, row.total_profit))

print("Data successfully inserted into sales_silver.")


Data successfully inserted into sales_silver.


In [31]:
session.execute("""
CREATE TABLE IF NOT EXISTS sales_by_region (
    region TEXT PRIMARY KEY,
    total_profit FLOAT
);
""")
print("Table sales_by_region created successfully.")



Table sales_by_region created successfully.


In [32]:
import pandas as pd

# Step 1: Load data from sales_bronze
query = "SELECT region, total_profit FROM sales_bronze"
rows = session.execute(query)

# Convert the data into a Pandas DataFrame for easy aggregation
df = pd.DataFrame(rows)

# Step 2: Perform the aggregation in Python
aggregated_data = df.groupby('region', as_index=False).agg({'total_profit': 'sum'})

# Step 3: Insert aggregated data into the sales_by_region table
for index, row in aggregated_data.iterrows():
    session.execute("""
        INSERT INTO sales_by_region (region, total_profit)
        VALUES (%s, %s)
    """, (row['region'], row['total_profit']))

print("Data successfully inserted into sales_by_region.")


Data successfully inserted into sales_by_region.


In [34]:
session.execute("""
CREATE TABLE IF NOT EXISTS sales_by_region (
    region TEXT PRIMARY KEY,
    total_profit FLOAT
);
""")
print("Table sales_by_region created successfully.")




Table sales_by_region created successfully.


In [35]:
import pandas as pd

# Step 1: Load data from sales_bronze (assuming you have a 'sales_bronze' table)
query = "SELECT region, total_profit FROM sales_bronze"
rows = session.execute(query)

# Convert to Pandas DataFrame for easy aggregation
df = pd.DataFrame(rows)

# Step 2: Aggregate data by 'region'
aggregated_data = df.groupby('region', as_index=False).agg({'total_profit': 'sum'})

# Step 3: Insert the aggregated data into the sales_by_region table
for index, row in aggregated_data.iterrows():
    session.execute("""
        INSERT INTO sales_by_region (region, total_profit)
        VALUES (%s, %s)
    """, (row['region'], row['total_profit']))

print("Data successfully inserted into sales_by_region.")


Data successfully inserted into sales_by_region.


In [37]:
session.execute("""
CREATE TABLE IF NOT EXISTS sales_by_item_type (
    item_type TEXT PRIMARY KEY,
    total_units_sold INT
);
""")
print("Table sales_by_item_type created successfully.")



Table sales_by_item_type created successfully.


In [38]:
import pandas as pd

# Step 1: Load data from sales_bronze (assuming you have a 'sales_bronze' table)
query = "SELECT item_type, units_sold FROM sales_bronze"
rows = session.execute(query)

# Convert to Pandas DataFrame for easy aggregation
df = pd.DataFrame(rows)

# Step 2: Aggregate data by 'item_type'
aggregated_data = df.groupby('item_type', as_index=False).agg({'units_sold': 'sum'})

# Step 3: Insert the aggregated data into the sales_by_item_type table
for index, row in aggregated_data.iterrows():
    session.execute("""
        INSERT INTO sales_by_item_type (item_type, total_units_sold)
        VALUES (%s, %s)
    """, (row['item_type'], row['units_sold']))

print("Data successfully inserted into sales_by_item_type.")


Data successfully inserted into sales_by_item_type.


In [40]:
session.execute("""
CREATE TABLE IF NOT EXISTS monthly_sales_summary (
    order_date TEXT PRIMARY KEY,
    total_revenue FLOAT
);
""")
print("Table monthly_sales_summary created successfully.")



Table monthly_sales_summary created successfully.


In [41]:
import pandas as pd

# Step 1: Retrieve data from sales_bronze (assuming you have the 'sales_bronze' table)
query = "SELECT order_date, total_revenue FROM sales_bronze"
rows = session.execute(query)

# Convert to Pandas DataFrame for easy aggregation
df = pd.DataFrame(rows)

# Step 2: Aggregate data by 'order_date'
aggregated_data = df.groupby('order_date', as_index=False).agg({'total_revenue': 'sum'})

# Step 3: Insert the aggregated data into the monthly_sales_summary table
for index, row in aggregated_data.iterrows():
    session.execute("""
        INSERT INTO monthly_sales_summary (order_date, total_revenue)
        VALUES (%s, %s)
    """, (row['order_date'], row['total_revenue']))

print("Data successfully inserted into monthly_sales_summary.")


Data successfully inserted into monthly_sales_summary.


In [42]:
# Query Gold tables and print results
tables = ["sales_by_region", "sales_by_item_type", "monthly_sales_summary"]

for table in tables:
    print(f"Data from {table}:")
    rows = session.execute(f"SELECT * FROM {table};")
    for row in rows:
        print(row)


Data from sales_by_region:
Row(region='Australia and Oceania', total_profit=3486940.0)
Row(region='Europe', total_profit=11267281.0)
Row(region='Middle East and North Africa', total_profit=6514262.0)
Row(region='Central America and the Caribbean', total_profit=4252300.0)
Row(region='Asia', total_profit=6749896.0)
Row(region='Sub-Saharan Africa', total_profit=7651892.0)
Row(region='North America', total_profit=1404621.5)
Data from sales_by_item_type:
Row(item_type='Household', total_units_sold=57640)
Row(item_type='Office Supplies', total_units_sold=42814)
Row(item_type='Vegetables', total_units_sold=7368)
Row(item_type='Snacks', total_units_sold=14377)
Row(item_type='Personal Care', total_units_sold=39045)
Row(item_type='Meat', total_units_sold=50437)
Row(item_type='Fruits', total_units_sold=65920)
Row(item_type='Beverages', total_units_sold=45206)
Row(item_type='Cereal', total_units_sold=45776)
Row(item_type='Cosmetics', total_units_sold=65707)
Row(item_type='Baby Food', total_units_s

In [44]:
import pandas as pd

# Load the data
df = pd.read_csv('/content/sales_100 (1).csv')

# Convert the 'Order Date' and 'Ship Date' columns to datetime format
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce', dayfirst=True)
df['Ship Date'] = pd.to_datetime(df['Ship Date'], errors='coerce', dayfirst=True)

# Check the cleaned data
print(df[['Order Date', 'Ship Date']].head())


  Order Date  Ship Date
0 2012-07-27 2012-07-28
1 2013-09-14 2013-10-19
2 2015-05-15 2015-06-04
3 2017-05-17 2017-07-02
4 2016-10-26 2016-12-04


<ipython-input-44-7b0f39de87eb>:7: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce', dayfirst=True)
<ipython-input-44-7b0f39de87eb>:8: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['Ship Date'] = pd.to_datetime(df['Ship Date'], errors='coerce', dayfirst=True)


In [48]:
# Query for Bronze table
bronze_query = "SELECT * FROM sales_bronze;"
bronze_data = session.execute(bronze_query)

print("------ Bronze Table Data ------")
for row in bronze_data:
    print(row)

# Query for Silver table
silver_query = "SELECT * FROM sales_silver;"
silver_data = session.execute(silver_query)

print("\n------ Silver Table Data ------")
for row in silver_data:
    print(row)




------ Bronze Table Data ------
Row(order_id=UUID('d05750ff-3595-4aeb-a0c6-25bc2f0610d3'), country='Uganda', item_type='Cosmetics', order_date='2/28/2015', order_priority='M', region='Sub-Saharan Africa', sales_channel='Online', ship_date='3/15/2015', total_cost=1588143.25, total_profit=1048610.0, total_revenue=2636753.25, unit_cost=263.3299865722656, unit_price=437.20001220703125, units_sold=6031)
Row(order_id=UUID('e0620cbf-ae2c-4e26-adec-57b73ea99614'), country='Burundi', item_type='Beverages', order_date='3/9/2011', order_priority='H', region='Sub-Saharan Africa', sales_channel='Online', ship_date='4/9/2011', total_cost=112314.0703125, total_profit=55326.78125, total_revenue=167640.84375, unit_cost=31.790000915527344, unit_price=47.45000076293945, units_sold=3533)
Row(order_id=UUID('f98b5059-660a-4e1d-b38e-95549e6cfd1d'), country='Tanzania', item_type='Cosmetics', order_date='5/23/2016', order_priority='L', region='Sub-Saharan Africa', sales_channel='Offline', ship_date='5/24/2016'

# New Section